Устанавливаем и импортируем необходимые библиотеки

In [1]:
!pip3 install torch

   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/203.0 MB 2.4 MB/s eta 0:01:25
   ---------------------------------------- 1.3/203.0 MB 2.2 MB/s eta 0:01:31
   ---------------------------------------- 1.8/203.0 MB 2.3 MB/s eta 0:01:26
   ---------------------------------------- 2.4/203.0 MB 2.4 MB/s eta 0:01:24
    --------------------------------------- 2.6/203.0 MB 2.4 MB/s eta 0:01:25
    --------------------------------------- 2.9/203.0 MB 2.2 MB/s eta 0:01:34
    --------------------------------------- 3.7/203.0 MB 2.3 MB/s eta 0:01:26
    --------------------------------------- 4.2/203.0 MB 2.3 MB/s eta 0:01:27
    --------------------------------------- 4.7/203.0 MB 2.3 MB/s eta 0:01:25
    --------------------------------------- 4.7/203.0 MB 2.3 MB/s eta 0:01:25
    --------------------------------------- 5.0/203.0 MB 2.1 MB/s eta 0:01:34


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections.abc import Iterable
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable

In [ ]:
import torch
import torch.nn
import torch.nn.functional as F
import torchvision.models as models

Задаём [параметры](https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2) для нормализации входных изображений

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [ ]:
def save_adversarial_image(x_adv, filename, resized):
    x_adv = reverse_transform(x_adv)
    im = Image.fromarray(x_adv)
    im2 = resized.copy()
    im2.paste(im, (16, 16))
    im2.save(filename)

Преобразуем изображение в формате tensor в изображение в формате numpy array

In [ ]:
def reverse_transform(image_tensor):
    x = image_tensor.squeeze(0)
    # reverse of normalization op
    x = x.mul(torch.FloatTensor(std).view(3, 1, 1)).add(torch.FloatTensor(mean).view(3, 1, 1)).numpy()
    x = np.transpose(x, (1, 2, 0))  # C X H X W  ==>   H X W X C
    x = np.clip(x, 0, 1)
    x *= 255
    return x.astype(np.uint8)

Сбрасываем [градиенты]() тензора

In [ ]:
def zero_gradients(x):
    if isinstance(x, torch.Tensor):
        if x.grad is not None:
            x.grad.detach_()
            x.grad.zero_()
    elif isinstance(x, Iterable):
        for elem in x:
            zero_gradients(elem)

Визуализируем результат проведённой атаки.

На вход подаются:
- Исходное изображение
- Adversarial изображение
- Параметр epsilon, определяющий величину вносимых возмущений
- Предсказание для исходного изображения
- Предсказание для adversarial изображения
- Точность предсказания для исходного изображения
- Точность предсказания для adversarial изображения



In [ ]:
def visualize(x, x_adv, x_grad, epsilon, clean_pred, adv_pred, clean_prob, adv_prob):
    x = reverse_transform(x)
    x_adv = reverse_transform(x_adv)
    x_grad = reverse_transform(x_grad)
    figure, ax = plt.subplots(1, 3, figsize=(18, 8))
    ax[0].imshow(x)
    ax[0].set_title('Clean Example', fontsize=20)
    ax[1].imshow(x_grad)
    ax[1].set_title('Perturbation', fontsize=20)
    ax[1].set_yticklabels([])
    ax[1].set_xticklabels([])
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[2].imshow(x_adv)
    ax[2].set_title('Adversarial Example', fontsize=20)
    ax[0].axis('off')
    ax[2].axis('off')
    ax[0].text(1.1, 0.5, "+{}*".format(round(epsilon, 3)), size=15, ha="center",
               transform=ax[0].transAxes)
    ax[0].text(0.5, -0.13, "Prediction: {}\n Probability: {}".format(clean_pred, clean_prob), size=15, ha="center",
               transform=ax[0].transAxes)
    ax[1].text(1.1, 0.5, " = ", size=15, ha="center", transform=ax[1].transAxes)
    ax[2].text(0.5, -0.13, "Prediction: {}\n Probability: {}".format(adv_pred, adv_prob), size=15, ha="center",
               transform=ax[2].transAxes)
    plt.show()

Функция для генерации состязательного изображения.

На вход подаются:
- Исходное изображение
- ID целевого класса

In [ ]:
def generate_adv(input_image, target_class_id, categories):
  # Имя файла для сохранения состязательного изображения
  output_filename = '/content/sample_data/adv'+ str(target_class_id) + '.png'

  # Загружаем и обрабатываем исходное изображение
  img_base = Image.open(...)
  resized = transforms.Resize(256).forward(...)
  preprocess = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean, std)
  ])

  # Предсказание для исходного изображения
  model_target = models.mobilenet_v2("IMAGENET1K_V2") # Зададим модель, которую необходимо атаковать - целевую модель
  model_target.eval()
  with torch.no_grad():
      output = model_target() # Сделаем предсказание целевой моделью для поданного на вход изображение (преобразованного в тензор)
  probabilities = F.softmax(output[0], dim=0)
  top_prob, top_catid = torch.topk(probabilities, 1)
  print('Оригинальное изображение')
  print('---------------')
  for i in range(top_prob.size(0)):
      print(categories[top_catid[i]], top_prob[i].item())

  # Генерируем состязательное изображение
  image_tensor = preprocess(...) # Преобразуем в тензор
  image_tensor = image_tensor.unsqueeze(0)  # Добавим +1 к размерности тензора  C x H x W ==> B x C x H x W
  img_variable = Variable(..., requires_grad=True)  # Конвертируем image tensor в тип variable
  y_target = Variable(torch.LongTensor([target_class_id]), requires_grad=False) # Конвертируем target class tensor в тип variable
  # Задаём параметры для нашей атаки
  epsilon = ...
  num_steps = ...
  alpha = ...
  for i in range(num_steps):
      zero_gradients(img_variable)
      output = model_target.forward(img_variable)
      # Задаём и вычисляем функцию потерь
      loss = ...
      loss_cal = ...
      loss_cal.backward()
      # Вычисляем градиент и получаем состязательное изображение
      x_grad = ...
      adv_temp = ...
      total_grad = ...
      total_grad = ...
      x_adv = image_tensor + ...
      img_variable.data = x_adv

  # Обрабатываем состязательное изображение, получаем предсказание для него и точность предсказания, затем визуализируем результат
  output_adv = model_target.forward(...)
  x_adv_pred = categories[torch.max(output_adv.data, 1)[1][0].item()]
  output_adv_probs = F.softmax(output_adv, dim=1)
  x_adv_pred_prob = round((torch.max(output_adv_probs.data, 1)[0][0]).item() * 100, 4)
  x_pred = categories[top_catid[0]]
  x_pred_prob = round(top_prob[0].item() * 100, 4)
  visualize(image_tensor, img_variable.data, total_grad, epsilon, x_pred, x_adv_pred, x_pred_prob,
                x_adv_pred_prob)
  save_adversarial_image(x_adv, output_filename, resized)

Классы для набора данных Imagenet можно скачать [отсюда](https://github.com/pytorch/hub/blob/master/imagenet_classes.txt)

In [ ]:
file_labels = '/content/sample_data/......txt'
with open(file_labels, "r") as f:
    categories = [s.strip() for s in f.readlines()]

Запустим генерацию состязательного изображения

In [ ]:
input_image = '/content/sample_data/.......png'
target_class_id = .....

generate_adv(input_image, target_class_id, categories)